# Dataset Labeling

This is a **manual workflow**. Once a dataset has been populated with the results from the **Ensemble Exploration** workflow, this workflow can be used to select the samples to be labeled. Here you manually select the final images to be labeled. These will be exported and are ready to be labeled in the second part of the notebbok, where CVAT is used for that purpose.


In [ ]:
# Connect to CVAT
# Add your credentials to the .secret file. Follow these instructions: https://docs.voxel51.com/integrations/cvat.html#authentication")
import os

with open("../.secret", "r") as file:
    for line in file:
        key, value = line.strip().split("=")
        if key in [
            "FIFTYONE_CVAT_USERNAME",
            "FIFTYONE_CVAT_PASSWORD",
            "FIFTYONE_CVAT_EMAIL",
        ]:
            print(f"Key {key} found")
            os.environ[key] = value

In [ ]:
from fiftyone import ViewField as F
import fiftyone as fo
import fiftyone.utils.cvat as fouc

In [ ]:
# Loading dataset and filtering view for optimized performance
# Shows only samples with min. n unique detections from Ensemble Exploration
# Shows only labels that belong to the unique detections

configuration = {
    "dataset_name": "fisheye8k",  # Dataset to load
    "min_n_unique_selection": 20,  # Focus on samples with many unique instances
}

ensemble_selection_field = "n_unique_ensemble_selection"
ensemble_selection_tag = "detections_overlap"
dataset_name = configuration["dataset_name"]
dataset = fo.load_dataset(dataset_name)
min_n_unique_selection = configuration["min_n_unique_selection"]
view_n_unique_exploration = dataset.match(
    F(ensemble_selection_field) >= min_n_unique_selection
)
view_tagged_labels = view_n_unique_exploration.select_labels(
    tags=ensemble_selection_tag
)

In [ ]:
# Use this view to label the samples you want to have labeled
# Choose the tag_for_labeling to tag the samples you want to label
tag_for_labeling = "todo_labeling"
fo.launch_app(view=view_tagged_labels)

In [ ]:
# Inspect your selection of samples to be labeled
view_selection = dataset.match_tags(tag_for_labeling)
print(view_selection)

In [ ]:
# Export the dataset you want to have labeled
# This example uses the YOLOv5Dataset format which also exports labels
# https://docs.voxel51.com/user_guide/export_datasets.html#yolov5dataset

splits = ["train", "val"]  # Select all the splits your dataset has
export_dir = "/media/dbogdoll/Datasets/vru_labeling"  # Directory to export the files to
label_field = "pred_google_owlv2_large_patch14_finetuned"  # Include a prediction field from one of the zero-shot models

classes = view_selection.distinct(
    f"{label_field}.detections.label"
)  # Get all the classes from the prediction field

# Export the splits
for split in splits:
    split_view = view_selection.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field=label_field,
        split=split,
        classes=classes,
    )

In [ ]:
# If you want to label the dataset in-house, this example shows how to use CVAT. Voxel51 also has other integrations
# You will only need a CVAT account and set the environment variables at the top of the notebook
# https://docs.voxel51.com/integrations/cvat.html

classes_for_labeling = [
    "motorbike/cycler",
    "pedestrian",
]  # ["car", "truck", "bus", "trailer", "motorbike/cycler", "pedestrian", "van", "pickup"] Classes from Mcity Fisheye dataset
label_schema = {
    "new_ground_truth": {
        "type": "polylines",
        "classes": classes_for_labeling,
    },
}

anno_key = "cvat"
# if view_selection.has_annotation_runs:
#    view_selection.delete_annotation_run(anno_key)

In [ ]:
view_selection.annotate(
    anno_key,
    backend="cvat",
    label_schema=label_schema,
    launch_editor=True,
)

In [ ]:
view_selection.list_annotation_runs()
info = dataset.get_annotation_info(anno_key)
print(info)

In [ ]:
# Once you have annotated the dataset, you can import the labels back into FiftyOne
dataset.load_annotations(anno_key)
view_annotations = dataset.load_annotation_view(anno_key)
fo.launch_app(view=view_annotations)

In [ ]:
# Import a labeled dataset from CVAT if the annotation_key was deleted

cvat_project_name = "FiftyOne_mcity_fisheye_3_months"

dataset_labeled = fo.Dataset()
fouc.import_annotations(
    dataset,
    project_name=cvat_project_name,
    data_path="/tmp/cvat_import",
    download_media=True,
)

session = fo.launch_app(dataset_labeled)

In [ ]:
# If dataset got extended, merge into existing dataset
# Add tags prior to merging
# for sample in dataset_labeled:
#    sample.tags = ["addition_2025_01_13","train"]
#    sample.save()
# dataset_exitsting.merge_samples(dataset_labeled)

In [ ]:
# Cleanup

# Delete tasks from CVAT
# results = dataset.load_annotation_results(anno_key)
# results.cleanup()

# Delete run record (not the labels) from FiftyOne
# dataset.delete_annotation_run(anno_key)